# Code

In [13]:
import numpy as np
import numba
from matplotlib import pyplot as plt

Numba-jitted functions to work with polynomials

In [14]:
@numba.njit(cache=True)
def identity_separator(x):
    return np.array([0,len(x),1])

@numba.njit(cache=True)
def separator_size(separator):
    return int(np.ceil((separator[1]-separator[0])/separator[2]))

@numba.njit(cache=True,parallel=True)
def get_terms_from_separator(coefs, separator):
    start = separator[0]
    end = separator[1]
    step = separator[2]
    return coefs[np.arange(start,end,step)]

@numba.njit(cache=True)
def get_even_separator(coefs):
    return np.array([0,len(coefs),2])

@numba.njit(cache=True)
def get_odd_separator(coefs):
    return np.array([1,len(coefs),2])

@numba.njit(cache=True)
def even_separator(separator):
    return np.array([separator[0],separator[1],separator[2]*2])

@numba.njit(cache=True)
def odd_separator(separator):
    return np.array([separator[0]+separator[2],separator[1],separator[2]*2])

@numba.njit(cache=True,fastmath=True,parallel=True)
def evaluate_polynomial(coefs,x):
    y = np.zeros(x.shape, dtype=float)
    coefs_len = len(coefs)-1
    for i in range(len(coefs)):
        y *= x
        y += coefs[coefs_len-i]
    return y

@numba.njit(cache=True)
def evaluate_polynomial_fft(coefs,x):
    return __evaluate_polynomial_fft(coefs,x,identity_separator(coefs))

@numba.njit(cache=True,fastmath=True, parallel=True)
def __evaluate_polynomial_fft(coefs,x,separator):
    # use upper-bound on recursion, to use base polynomial evaluation
    # this value is found by hands
    if separator_size(separator)<=1024: 
        separated_coefs = get_terms_from_separator(coefs,separator)
        return evaluate_polynomial(separated_coefs,x)
    evens = even_separator(separator)
    odds = odd_separator(separator)
    x2 = x*x
    # these two could be forked
    even_sum = __evaluate_polynomial_fft(coefs,x2,evens)
    odd_sum  = x*__evaluate_polynomial_fft(coefs,x2,odds)
    return even_sum+odd_sum

@numba.njit(cache=True,fastmath=True)
def get_even_terms(coefs):
    evens = np.arange((len(coefs)+1)//2)*2
    return coefs[evens]

@numba.njit(cache=True,fastmath=True)
def get_odd_terms(coefs):
    odds = np.arange(len(coefs)//2)*2+1
    return coefs[odds]

In [15]:
class Polynomial:
    def __init__(self, coefficients) -> None:
        self.coefficients=coefficients

    def evaluate(self,x):
        return evaluate_polynomial(self.coefficients,x)
    
    def evaluate_fft(self,x):
        return evaluate_polynomial_fft(self.coefficients,x)
    

Showcase of how used separators objects are working.
They used to split data on even and odd indices with deep hierarchy, so if we need to get elements of array with even indices and then get elements with odd indices, we can use three numbers (start,end,step) to describe such behavior.

This division is used for fft-based approach for evaluating polynomial

In [16]:
a = np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16])

even = get_even_terms(a)
odd = get_odd_terms(a)

even_even = get_even_terms(even)
even_odd = get_odd_terms(even)
odd_even = get_even_terms(odd)
odd_odd = get_odd_terms(odd)

even_even_odd = get_odd_terms(even_even)
odd_even_odd = get_odd_terms(odd_even)
even_odd_even = get_even_terms(even_odd)

print("orig\t\t",a)
print("even\t\t",even)
print("even_even\t",even_even)
print("even_odd\t",even_odd)

print("odd\t\t",odd)
print("odd_even\t",odd_even)
print("odd_odd\t\t",odd_odd)

print("even_even_odd\t",even_even_odd)
print("odd_even_odd\t",odd_even_odd)
print("even_odd_even\t",even_odd_even)

orig		 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
even		 [ 0  2  4  6  8 10 12 14 16]
even_even	 [ 0  4  8 12 16]
even_odd	 [ 2  6 10 14]
odd		 [ 1  3  5  7  9 11 13 15]
odd_even	 [ 1  5  9 13]
odd_odd		 [ 3  7 11 15]
even_even_odd	 [ 4 12]
odd_even_odd	 [ 5 13]
even_odd_even	 [ 2 10]


In [17]:
a = np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16])

identity = identity_separator(a)
even = even_separator(identity)
odd = odd_separator(identity)

even_even = even_separator(even)
even_odd = odd_separator(even)
odd_even = even_separator(odd)
odd_odd = odd_separator(odd)

even_even_odd = odd_separator(even_even)
odd_even_odd = odd_separator(odd_even)
even_odd_even = even_separator(even_odd)

print("identity\t",get_terms_from_separator(a,identity))
print("even\t\t",get_terms_from_separator(a,even))
print("even_even\t",get_terms_from_separator(a,even_even))
print("even_odd\t",get_terms_from_separator(a,even_odd))

print("odd\t\t",get_terms_from_separator(a,odd))
print("odd_even\t",get_terms_from_separator(a,odd_even))
print("odd_odd\t\t",get_terms_from_separator(a,odd_odd))

print("even_even_odd\t",get_terms_from_separator(a,even_even_odd))
print("odd_even_odd\t",get_terms_from_separator(a,odd_even_odd))
print("even_odd_even\t",get_terms_from_separator(a,even_odd_even))

separator_size(identity)

identity	 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
even		 [ 0  2  4  6  8 10 12 14 16]
even_even	 [ 0  4  8 12 16]
even_odd	 [ 2  6 10 14]
odd		 [ 1  3  5  7  9 11 13 15]
odd_even	 [ 1  5  9 13]
odd_odd		 [ 3  7 11 15]
even_even_odd	 [ 4 12]
odd_even_odd	 [ 5 13]
even_odd_even	 [ 2 10]


17

# Showcase

In [23]:
x = np.linspace(-1,1,100000)
coefs = np.random.normal(0,1,90000)
pol = Polynomial(coefs)

In [24]:
y1=pol.evaluate(x)

In [25]:
y2=pol.evaluate_fft(x)

In [27]:
print(np.max(np.abs(y1-y2)))
print(np.max(np.abs(y2-y3)))

9.109868415180244e-11
9.109868415180244e-11
